In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from nltk.tokenize import sent_tokenize
from keras import layers, models, optimizers
import csv



In [3]:
import pandas as pd
import numpy as np

In [4]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import random

[nltk_data] Downloading package punkt to /Users/james/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
csv.field_size_limit(999999)
train = pd.read_csv('raw_data/fulltrain.csv', header = None, names=['class','text'])
test = pd.read_csv("raw_data/balancedtest.csv", header = None, names=['class','text'])

In [6]:
train['class'].value_counts()

class
3    17870
1    14047
4     9995
2     6942
Name: count, dtype: int64

In [7]:
one = train.loc[train['class'] == 1]
two = train.loc[train['class'] == 2]
three = train.loc[train['class'] == 3]
four = train.loc[train['class'] == 4]

In [8]:
train = pd.concat([train, one.sample(n=3953), two.sample(n=6000), two.sample(n=5058), three.sample(n=130), four.sample(n=8005)], ignore_index=True)


In [9]:
train['class'].value_counts()

class
1    18000
2    18000
3    18000
4    18000
Name: count, dtype: int64

In [10]:
x_train = train['text']
x_test = test['text']

In [11]:
tokenizer = Tokenizer(num_words=700)
tokenizer.fit_on_texts(x_train.values)
word_index = tokenizer.word_index

# Convert texts to sequences
train_sequences = tokenizer.texts_to_sequences(x_train)
test_sequences = tokenizer.texts_to_sequences(x_test)

# Padding
padding_type = 'post'
truncating_type = 'post'

In [12]:
vocab_size = len(word_index) + 1
max_length = 100

X_train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=truncating_type)
X_test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=truncating_type)


In [13]:
vocab_size = len(word_index) + 1                          

maxlen = 100

X_train = pad_sequences(train_sequences, padding='post', maxlen=maxlen)
X_test = pad_sequences(test_sequences, padding='post', maxlen=maxlen)

In [25]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from tensorflow.keras.layers import Embedding

from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Concatenate




In [17]:
def create_embedding_matrix(word_vectors, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, idx in word_index.items():
        if word in word_vectors:
            embedding_matrix[idx] = word_vectors[word]
    return embedding_matrix

# Load Word2Vec KeyedVectors
word2vec_path = 'word2vec-google-news-300.model'  # Adjust as needed
word2vec = KeyedVectors.load(word2vec_path)

# Create the embedding matrix for your tokenizer's vocabulary
embedding_dim = 300  # or the dimensions of your word vectors
embedding_matrix = create_embedding_matrix(word2vec, tokenizer.word_index, embedding_dim)



In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
tfidf_vectorizer = TfidfVectorizer(max_features=10000)  # Adjust the number of max_features as needed
tfidf_matrix = tfidf_vectorizer.fit_transform(x_train)  # Replace 'your_text_data' with your actual text data
tfidf_feature_length = tfidf_matrix.shape[1]  # Number of TF-IDF features

In [21]:
y_train = train['class'] - 1
y_test = test['class'] - 1

In [22]:
num_classes = 4
y_train_categorical = to_categorical(y_train, num_classes=num_classes)
y_test_categorical = to_categorical(y_test, num_classes=num_classes)

In [23]:
y_test_categorical

array([[1., 0., 0., 0.],
       [1., 0., 0., 0.],
       [1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.]])

In [26]:
sequence_input = Input(shape=(max_length,), dtype='int32')
tfidf_input = Input(shape=(tfidf_feature_length,))

In [31]:
import tensorflow as tf
embedding_layer = Embedding(input_dim=embedding_matrix.shape[0],  # Vocabulary size
                            output_dim=embedding_matrix.shape[1],  # Embedding dimensionality
                            embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),  # Use pre-trained weights
                            input_length=max_length,  # Length of input sequences
                            trainable=False)(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedding_layer)
x = GlobalMaxPooling1D()(x)
x = Dense(32, activation='relu')(x)
cnn_path = Dropout(0.5)(x)


In [32]:
# TF-IDF path
y = Dense(128, activation='relu')(tfidf_input)
tfidf_path = Dropout(0.5)(y)


In [39]:
from tensorflow.keras.layers import Concatenate

# Assuming cnn_path and tfidf_path are defined in your model as outputs from previous layers
combined = Concatenate()([cnn_path, tfidf_path])

In [40]:
# Output layer
output = Dense(4, activation='softmax')(combined)

# Create and compile the model
model = Model(inputs=[sequence_input, tfidf_input], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-3), metrics=['accuracy'])


In [41]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 100)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 100, 300)  │ 77,786,100 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 96, 128)   │    192,128 │ embedding_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 128)       │          0 │ conv1d_2[0][0]    │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 10000)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 32)        │      4,128 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │  1,280,128 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 32)        │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 160)       │          0 │ dropout[0][0],    │
│ (Concatenate)       │                   │            │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 4)         │        644 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 79,263,128 (302.36 MB)

 Trainable params: 1,477,028 (5.63 MB)

 Non-trainable params: 77,786,100 (296.73 MB)

In [43]:
X_train_sequences = tokenizer.texts_to_sequences(x_train)  # Replace with your training text data
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_length)
X_train_tfidf = tfidf_vectorizer.transform(x_train).toarray()  # Replace with your training text data


In [46]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min', restore_best_weights=True)


In [47]:
model.fit([X_train_padded, X_train_tfidf], y_train_categorical, batch_size=128, epochs=10, validation_split=0.1, callbacks=[early_stopping])


Epoch 1/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 19s 37ms/step - accuracy: 0.9460 - loss: 0.1947 - val_accuracy: 0.9833 - val_loss: 0.0740
Epoch 2/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 18s 36ms/step - accuracy: 0.9904 - loss: 0.0413 - val_accuracy: 0.9932 - val_loss: 0.0417
Epoch 3/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 18s 36ms/step - accuracy: 0.9961 - loss: 0.0195 - val_accuracy: 0.9990 - val_loss: 0.0105
Epoch 4/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 18s 36ms/step - accuracy: 0.9983 - loss: 0.0101 - val_accuracy: 0.9997 - val_loss: 0.0048
Epoch 5/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 19s 38ms/step - accuracy: 0.9992 - loss: 0.0067 - val_accuracy: 0.9999 - val_loss: 0.0024
Epoch 6/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 19s 38ms/step - accuracy: 0.9992 - loss: 0.0047 - val_accuracy: 0.9999 - val_loss: 0.0022
Epoch 7/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 21s 41ms/step - accuracy: 0.9994 - loss: 0.0036 - val_accuracy: 0.9999 - val_loss: 0.0013
Epoch 8/10
507/507 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - accuracy: 0.9997 - loss: 0.0029 - 

In [49]:
X_test_sequences = tokenizer.texts_to_sequences(x_test)  # Replace with your training text data
X_test_padded = pad_sequences(X_test_sequences, maxlen=max_length)
X_test_tfidf = tfidf_vectorizer.transform(x_test).toarray() 

In [51]:
y_pred = model.predict([X_test_padded, X_test_tfidf])
y_pred_classes = y_pred.argmax(axis=1)

94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [52]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

In [53]:
score = f1_score(y_test, y_pred_classes, average='macro')
print('Score on validation = {}'.format(score))

Score on validation = 0.6909470184412403


In [54]:
print('Classification report on test data:')
# Make sure y_test is in the correct format for comparison
print(classification_report(y_test, y_pred_classes))

Classification report on test data:
              precision    recall  f1-score   support

           0       0.86      0.73      0.79       750
           1       0.75      0.39      0.51       750
           2       0.55      0.74      0.63       750
           3       0.74      0.93      0.83       750

    accuracy                           0.70      3000
   macro avg       0.72      0.70      0.69      3000
weighted avg       0.72      0.70      0.69      3000

